In [1]:
import random
from datetime import datetime, timedelta
import sqlite3

In [2]:
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

In [3]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Product (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    category TEXT,
    discontinued BOOLEAN
)''')


In [4]:

cursor.execute('''
CREATE TABLE IF NOT EXISTS Variant (
    variant_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER,
    name TEXT,
    price REAL,
    discontinued BOOLEAN,
    FOREIGN KEY (product_id) REFERENCES Product (product_id)
)''')


In [5]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customer (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT,
    phone TEXT,
    address TEXT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
)''')


In [6]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS "OrderItem" (
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER,
    variant_id INTEGER,
    quantity INTEGER,
    price_at_purchase REAL,
    FOREIGN KEY (order_id) REFERENCES "Order" (order_id),
    FOREIGN KEY (variant_id) REFERENCES Variant (variant_id)
)''')

In [7]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS "Order" (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    order_date TIMESTAMP,
    total_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES Customer (customer_id)
)''')

In [8]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS CustomerAddress (
    address_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    address TEXT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY (customer_id) REFERENCES Customer (customer_id)
)''')

In [9]:
categories = ['clothing', 'groceries', 'electronics']
products = [
    {'name': 'T-Shirt', 'category': 'clothing'},
    {'name': 'Jeans', 'category': 'clothing'},
    {'name': 'Laptop', 'category': 'electronics'},
    {'name': 'Headphones', 'category': 'electronics'},
    {'name': 'Bread', 'category': 'groceries'},
    {'name': 'Milk', 'category': 'groceries'},
    {'name': 'Smartphone', 'category': 'electronics'},
    {'name': 'Jacket', 'category': 'clothing'},
    {'name': 'Cereal', 'category': 'groceries'},
    {'name': 'TV', 'category': 'electronics'}
]



In [10]:
customers = [
    {'name': 'Alice', 'email': 'alice@example.com', 'phone': '1234567890', 'address': '123 Main St'},
    {'name': 'Bob', 'email': 'bob@example.com', 'phone': '2345678901', 'address': '456 Elm St'},
    {'name': 'Charlie', 'email': 'charlie@example.com', 'phone': '3456789012', 'address': '789 Oak St'},
    {'name': 'David', 'email': 'david@example.com', 'phone': '4567890123', 'address': '101 Pine St'},
    {'name': 'Eve', 'email': 'eve@example.com', 'phone': '5678901234', 'address': '202 Maple St'},
    {'name': 'Frank', 'email': 'frank@example.com', 'phone': '6789012345', 'address': '303 Birch St'},
    {'name': 'Grace', 'email': 'grace@example.com', 'phone': '7890123456', 'address': '404 Cedar St'},
    {'name': 'Heidi', 'email': 'heidi@example.com', 'phone': '8901234567', 'address': '505 Spruce St'},
    {'name': 'Ivan', 'email': 'ivan@example.com', 'phone': '9012345678', 'address': '606 Fir St'},
    {'name': 'Judy', 'email': 'judy@example.com', 'phone': '1234567899', 'address': '707 Willow St'}
]


In [11]:
for product in products:
    cursor.execute('INSERT INTO Product (name, category, discontinued) VALUES (?, ?, ?)',
                   (product['name'], product['category'], False))

In [12]:
variants = [
    {'product_id': 1, 'name': 'Red', 'price': 10.0, 'discontinued': False},
    {'product_id': 1, 'name': 'Green', 'price': 12.0, 'discontinued': False},
    {'product_id': 2, 'name': 'Blue', 'price': 15.0, 'discontinued': False},
    {'product_id': 3, 'name': '16GB RAM', 'price': 1000.0, 'discontinued': False},
    {'product_id': 3, 'name': '8GB RAM', 'price': 900.0, 'discontinued': False},
    {'product_id': 4, 'name': 'Wireless', 'price': 50.0, 'discontinued': False},
    {'product_id': 5, 'name': 'Whole Grain', 'price': 3.0, 'discontinued': False},
    {'product_id': 6, 'name': 'Organic', 'price': 4.0, 'discontinued': False},
    {'product_id': 7, 'name': '64GB', 'price': 700.0, 'discontinued': False},
    {'product_id': 8, 'name': 'Large', 'price': 60.0, 'discontinued': False},
    {'product_id': 9, 'name': 'Honey Nut', 'price': 5.0, 'discontinued': False},
    {'product_id': 10, 'name': '4K', 'price': 500.0, 'discontinued': False}
]

In [13]:
for variant in variants:
    cursor.execute('INSERT INTO Variant (product_id, name, price, discontinued) VALUES (?, ?, ?, ?)',
                   (variant['product_id'], variant['name'], variant['price'], variant['discontinued']))

In [14]:
for customer in customers:
    created_at = datetime.now() - timedelta(days=random.randint(30, 365*2))
    updated_at = datetime.now()
    cursor.execute('INSERT INTO Customer (name, email, phone, address, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?)',
                   (customer['name'], customer['email'], customer['phone'], customer['address'], created_at, updated_at))

In [15]:
for _ in range(100):  
    customer_id = random.randint(1, len(customers))
    order_date = datetime.now() - timedelta(days=random.randint(1, 365*2))
    total_amount = 0
    cursor.execute('INSERT INTO "Order" (customer_id, order_date, total_amount) VALUES (?, ?, ?)',
                   (customer_id, order_date, total_amount))
    order_id = cursor.lastrowid
    
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        variant_id = random.randint(1, len(variants))
        quantity = random.randint(1, 3)
        price_at_purchase = cursor.execute('SELECT price FROM Variant WHERE variant_id = ?', (variant_id,)).fetchone()[0]
        total_amount += price_at_purchase * quantity
        cursor.execute('INSERT INTO OrderItem (order_id, variant_id, quantity, price_at_purchase) VALUES (?, ?, ?, ?)',
                       (order_id, variant_id, quantity, price_at_purchase))
    
    cursor.execute('UPDATE "Order" SET total_amount = ? WHERE order_id = ?', (total_amount, order_id))

In [16]:
for customer_id in range(1, len(customers) + 1):
    address = cursor.execute('SELECT address FROM Customer WHERE customer_id = ?', (customer_id,)).fetchone()[0]
    created_at = datetime.now() - timedelta(days=random.randint(30, 365*2))
    updated_at = datetime.now()
    cursor.execute('INSERT INTO CustomerAddress (customer_id, address, created_at, updated_at) VALUES (?, ?, ?, ?)',
                   (customer_id, address, created_at, updated_at))

In [17]:
conn.commit()
conn.close()

In [18]:
#Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months

In [19]:
conn = sqlite3.connect('ecommerce.db')  


query = '''
SELECT 
    c.customer_id, 
    c.name, 
    AVG(o.total_amount) AS average_order_amount
FROM 
    "Order" o
JOIN 
    Customer c ON o.customer_id = c.customer_id
WHERE 
    o.order_date >= DATE('now', '-6 months')
GROUP BY 
    c.customer_id, c.name
ORDER BY 
    average_order_amount DESC
LIMIT 5;
'''

In [20]:
import pandas as pd

In [21]:
df = pd.read_sql_query(query, conn)

In [22]:
df

,customer_id,name,average_order_amount
0,7,Grace,2692.000000
1,9,Ivan,2508.666667
2,4,David,2403.250000
3,5,Eve,1824.000000
4,6,Frank,1617.875000


In [23]:
#Retrieve the list of customer whose order value is lower this year as compared to previous year

In [24]:
query1 = '''
SELECT 
    c.customer_id, 
    c.name, 
    SUM(CASE WHEN strftime('%Y', o.order_date) = strftime('%Y', 'now') THEN o.total_amount ELSE 0 END) AS this_year,
    SUM(CASE WHEN strftime('%Y', o.order_date) = strftime('%Y', 'now', '-1 year') THEN o.total_amount ELSE 0 END) AS last_year
FROM 
    "Order" o
JOIN 
    Customer c ON o.customer_id = c.customer_id
GROUP BY 
    c.customer_id, c.name
HAVING 
    this_year < last_year;
'''

In [26]:
df1 = pd.read_sql_query(query1, conn)

In [27]:
df1

,customer_id,name,this_year,last_year
0,1,Alice,3215.0,12515.0
1,2,Bob,6502.0,15975.0
2,3,Charlie,7875.0,12887.0
3,5,Eve,7381.0,17478.0
4,6,Frank,12943.0,19357.0
5,7,Grace,10768.0,11807.0
6,8,Heidi,10344.0,22212.0
7,10,Judy,12722.0,15711.0


In [28]:
#Create a table showing cumulative purchase by a particular customer. Show the breakup of cumulative purchases by product category.

In [29]:
query2 = '''
SELECT 
    c.customer_id, 
    c.name, 
    p.category, 
    SUM(oi.quantity * oi.price_at_purchase) AS total_purchase
FROM 
    OrderItem oi
JOIN 
    "Order" o ON oi.order_id = o.order_id
JOIN 
    Customer c ON o.customer_id = c.customer_id
JOIN 
    Variant v ON oi.variant_id = v.variant_id
JOIN 
    Product p ON v.product_id = p.product_id
GROUP BY 
    c.customer_id, c.name, p.category
ORDER BY 
    c.customer_id, p.category;
'''

In [30]:
df2 = pd.read_sql_query(query2, conn)

In [31]:
df2

,customer_id,name,category,total_purchase
0,1,Alice,clothing,689.0
1,1,Alice,electronics,28000.0
2,1,Alice,groceries,118.0
3,2,Bob,clothing,929.0
4,2,Bob,electronics,30100.0
5,2,Bob,groceries,87.0
6,3,Charlie,clothing,552.0
7,3,Charlie,electronics,25650.0
8,3,Charlie,groceries,87.0
9,4,David,clothing,627.0


In [32]:
#Retrieve the list of top 5 selling products. Further bifurcate the sales by product variants

In [33]:
query3 = '''
SELECT 
    p.product_id, 
    p.name AS product_name, 
    v.name AS variant_name, 
    SUM(oi.quantity) AS total_quantity_sold
FROM 
    OrderItem oi
JOIN 
    Variant v ON oi.variant_id = v.variant_id
JOIN 
    Product p ON v.product_id = p.product_id
GROUP BY 
    p.product_id, p.name, v.variant_id, v.name
ORDER BY 
    total_quantity_sold DESC
LIMIT 5;
'''

In [34]:
df3 = pd.read_sql_query(query3, conn)

In [35]:
df3

,product_id,product_name,variant_name,total_quantity_sold
0,4,Headphones,Wireless,122
1,7,Smartphone,64GB,113
2,8,Jacket,Large,107
3,2,Jeans,Blue,100
4,3,Laptop,8GB RAM,99
